<a href="https://colab.research.google.com/github/Nourshaolin/Real-Time-Retail-Analytics-and-Product-Recommender/blob/main/RealTime_Retail_Analytics_and_Product_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Real-Time Retail Analytics and Product Recommender
Team members:
* Nour El Moujahid
* Hiba Jebbar
* Ilyass Louzali
* Aya Boumelha

##Milestone 1 (Nour):

Neural Network Workflow (PyTorch): Train deep learning models to predict
purchasing behavior and customer retention based on session logs and historical purchase
data.


###Part 1: Load Kaggle dataset into a Pandas dataframe
Kaggle dataset site: https://www.kaggle.com/datasets/lakshmi25npathi/online-retail-dataset

In [1]:
# Opendatasets is a Python library for downloading datasets from online sources like Kaggle
# Install the library
!pip install opendatasets --upgrade

In [2]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import opendatasets as od
import pandas as pd

In [3]:
# Download the dataset
# API TOKEN = {"username":"nourelmoujahid","key":"611189b7ac2a80012221c170415923b6"}
data_url = "https://www.kaggle.com/datasets/lakshmi25npathi/online-retail-dataset"
dataset = od.download(data_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: nourelmoujahid
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/online-retail-dataset


In [4]:
# Convert the downloaded dataset into a dataframe
file = 'online-retail-dataset/online_retail_II.xlsx'
df = pd.read_excel(file)

In [10]:
# Pandas dataframe loaded successfully
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Year,Month,Day,DayOfWeek,Hour
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,2009,12,1,1,7
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009,12,1,1,7
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009,12,1,1,7
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,2009,12,1,1,7
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,2009,12,1,1,7


###Part 2: Data Analysis

Features' Information:

* `InvoiceNo`: Invoice number. If this code starts with the letter 'c', it indicates a cancellation.

* `StockCode`: Product (item) code. A 5-digit integral number uniquely assigned to each distinct product.

* `Description`: Product (item) name.

* `Quantity`: The quantities of each product (item) per transaction.

* `InvoiceDate`: The day and time when a transaction was generated.

* `UnitPrice`: Product price per unit in sterling (Â£).

* `CustomerID`: Customer number. A 5-digit integral number uniquely assigned to each customer.

* `Country`: The name of the country where a customer resides.

Assumptions about the Features according Nour Ackerman

* `Invoice`: check number of 'c' per country or per `CustomerID`
* `StockCode`: meh, does not seem relevant. better check `Description`
* `Quantity`: important, how much you like the product per `Country` or `CustomerID`
* `InvoiceDate`: no clue my dude, I need to check
* `UnitPrice`: yea why not, how expensiveness affects purchases

In [ ]:
#features' data types
df.dtypes

,0
Invoice,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,datetime64[ns]
Price,float64
Customer ID,float64
Country,object


In [ ]:
# 525461 rows
# 8 columns
df.shape

(525461, 8)

In [ ]:
df.at[100434, "Country"]

'France'

In [ ]:
# Available countries
df["Country"].unique()

array(['United Kingdom', 'France', 'USA', 'Belgium', 'Australia', 'EIRE',
       'Germany', 'Portugal', 'Japan', 'Denmark', 'Nigeria',
       'Netherlands', 'Poland', 'Spain', 'Channel Islands', 'Italy',
       'Cyprus', 'Greece', 'Norway', 'Austria', 'Sweden',
       'United Arab Emirates', 'Finland', 'Switzerland', 'Unspecified',
       'Malta', 'Bahrain', 'RSA', 'Bermuda', 'Hong Kong', 'Singapore',
       'Thailand', 'Israel', 'Lithuania', 'West Indies', 'Lebanon',
       'Korea', 'Brazil', 'Canada', 'Iceland'], dtype=object)

In [ ]:
len(df["Description"].unique())

4682

In [ ]:
len(df["StockCode"].unique())

4632

In [ ]:
#Total quantity
df.groupby(["Country", "Description"])["Quantity"].sum().reset_index(name="TotalQuantity").sort_values(by="TotalQuantity", ascending=False)

,Country,Description,TotalQuantity
19033,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER,52252
19138,United Kingdom,WORLD WAR 2 GLIDERS ASSTD DESIGNS,51434
15369,United Kingdom,BROCADE RING PURSE,47495
14931,United Kingdom,ASSORTED COLOUR BIRD ORNAMENT,41855
17381,United Kingdom,PACK OF 72 RETRO SPOT CAKE CASES,36393
...,...,...,...
19218,United Kingdom,Zebra invcing error,-9000
19253,United Kingdom,ebay sales,-13630
19270,United Kingdom,missing,-15149
19260,United Kingdom,given away,-20000


##Milestone 2 (Hiba):
Similarity Search: Use high-dimensional similarity search to identify
customers with comparable behavior, enabling collaborative trends and tailored
suggestions.

# **Data Preparation**





In [5]:
# Inspect the data structure first
print(f"Dataset shape: {df.shape}")

Dataset shape: (525461, 8)


In [6]:
print("\nColumn names:")
print(df.columns.tolist())


Column names:
['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']


In [7]:
print("\nMissing values per column:")
print(df.isnull().sum())


Missing values per column:
Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64


In [8]:
# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [9]:
# Extract date components for time pattern analysis
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek  # 0=Monday, 6=Sunday
df['Hour'] = df['InvoiceDate'].dt.hour

In [11]:
# Filter out returns (negative quantities) and missing customer IDs
df_clean = df[(df['Quantity'] > 0) & (~df['Customer ID'].isna())]

In [12]:
print(df_clean[['Quantity', 'Price']].head())
print(df_clean[['Quantity', 'Price']].dtypes)


   Quantity  Price
0        12   6.95
1        12   6.75
2        12   6.75
3        48   2.10
4        24   1.25
Quantity      int64
Price       float64
dtype: object


In [13]:
df_clean = df[(df['Quantity'] > 0) & (~df['Customer ID'].isna())].copy()
df_clean['TotalAmount'] = df_clean['Quantity'] * df_clean['Price']


In [14]:
# Set the latest date in the dataset for recency calculation
latest_date = df_clean['InvoiceDate'].max()

In [15]:
print(f"\nData ranges from {df_clean['InvoiceDate'].min()} to {latest_date}")
print(f"\nAfter cleaning: {df_clean.shape} (rows, columns)")


Data ranges from 2009-12-01 07:45:00 to 2010-12-09 20:01:00

After cleaning: (407695, 14) (rows, columns)


In [16]:
# Show the cleaned data
df_clean.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Year,Month,Day,DayOfWeek,Hour,TotalAmount
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,2009,12,1,1,7,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009,12,1,1,7,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009,12,1,1,7,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,2009,12,1,1,7,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,2009,12,1,1,7,30.0



# 1. Feature Set 1: RFM (Recency, Frequency, Monetary) Metrics




Calculate RFM (Recency, Frequency, Monetary) metrics

RFM helps identify a customer's value based on:
- Recency: How recently they purchased (lower days = more engaged)
- Frequency: How often they purchase (higher = more loyal)bold text
- Monetary: How much they spend (higher = more valuable)


In [17]:
print("\nColumn names:")
print(df.columns.tolist())


Column names:
['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country', 'Year', 'Month', 'Day', 'DayOfWeek', 'Hour']


In [18]:
print(list(df_clean.columns))


['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country', 'Year', 'Month', 'Day', 'DayOfWeek', 'Hour', 'TotalAmount']


In [19]:
def calculate_rfm_metrics(df, latest_date):
    """
    Calculates RFM metrics for each customer.

    Purpose:
    - Recency identifies which customers are currently engaged
    - Frequency identifies loyal customers
    - Monetary identifies high-value customers
    - Average order value shows spending habits per transaction
    """
    # Group by customer on the cleaned dataset
    customer_group = df.groupby('Customer ID')

    # Recency = Days since last purchase
    recency = (latest_date - customer_group['InvoiceDate'].max()).dt.days.reset_index()
    recency.columns = ['Customer ID', 'Recency']

    # Frequency = Number of unique purchases
    frequency = customer_group['Invoice'].nunique().reset_index()
    frequency.columns = ['Customer ID', 'Frequency']

    # Monetary = Total spent
    monetary = customer_group['TotalAmount'].sum().reset_index()
    monetary.columns = ['Customer ID', 'Monetary']

    # Average order value
    avg_order = customer_group['TotalAmount'].mean().reset_index()
    avg_order.columns = ['Customer ID', 'AvgOrderValue']

    # Merge all metrics
    rfm = recency.merge(frequency, on='Customer ID')\
                 .merge(monetary, on='Customer ID')\
                 .merge(avg_order, on='Customer ID')

    return rfm
# Calculate RFM metrics
rfm_features = calculate_rfm_metrics(df_clean, latest_date)

# Display the RFM metrics for a few customers
print(f"RFM metrics calculated for {len(rfm_features)} customers")
rfm_features.head()

RFM metrics calculated for 4314 customers


,Customer ID,Recency,Frequency,Monetary,AvgOrderValue
0,12346.0,164,11,372.86,11.298788
1,12347.0,2,2,1323.32,18.638310
2,12348.0,73,1,222.16,11.108000
3,12349.0,42,3,2671.14,26.187647
4,12351.0,10,1,300.93,14.330000


# 2.   Feature Set 2: Time Patterns (Day/Hour of Purchases)









In [20]:
def calculate_time_patterns(df):
    """
    Extracts time-based purchase patterns for each customer.
    """
    # Create day of week dummies (0=Monday, 6=Sunday)
    dow_dummies = pd.get_dummies(df['DayOfWeek'], prefix='DayOfWeek')
    df_with_dow = pd.concat([df[['Customer ID', 'Invoice']], dow_dummies], axis=1)

    # Calculate average day of week distribution per customer
    dow_dist = df_with_dow.groupby('Customer ID')[dow_dummies.columns].mean().reset_index()

    # Group hours into time periods
    df['TimeOfDay'] = pd.cut(df['Hour'],
                             bins=[0, 6, 12, 18, 24],
                             labels=['Night', 'Morning', 'Afternoon', 'Evening'],
                             right=False)

    # Create time of day dummies
    tod_dummies = pd.get_dummies(df['TimeOfDay'], prefix='TimeOfDay')
    df_with_tod = pd.concat([df[['Customer ID', 'Invoice']], tod_dummies], axis=1)

    # Calculate average time of day distribution per customer
    tod_dist = df_with_tod.groupby('Customer ID')[tod_dummies.columns].mean().reset_index()

    # Create month dummies
    month_dummies = pd.get_dummies(df['Month'], prefix='Month')
    df_with_month = pd.concat([df[['Customer ID', 'Invoice']], month_dummies], axis=1)

    # Calculate average monthly distribution per customer
    month_dist = df_with_month.groupby('Customer ID')[month_dummies.columns].mean().reset_index()

    # Combine all time patterns
    time_patterns = dow_dist.merge(tod_dist, on='Customer ID')\
                            .merge(month_dist, on='Customer ID')

    return time_patterns

# Calculate time patterns
time_pattern_features = calculate_time_patterns(df_clean)
# Display the time patterns for a few customers
print(f"Time patterns extracted with {time_pattern_features.shape[1] - 1} features")
time_pattern_features.head()


Time patterns extracted with 23 features


,Customer ID,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,TimeOfDay_Night,TimeOfDay_Morning,...,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
0,12346.0,0.727273,0.151515,0.0,0.030303,0.090909,0.0,0.00000,0.0,0.212121,...,0.151515,0.00000,0.000000,0.575758,0.0,0.0,0.0,0.000000,0.0,0.151515
1,12347.0,0.000000,0.436620,0.0,0.000000,0.000000,0.0,0.56338,0.0,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.563380,0.0,0.436620
2,12348.0,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.0,0.0,1.0,0.000000,0.0,0.000000
3,12349.0,0.000000,0.009804,0.0,0.990196,0.000000,0.0,0.00000,0.0,0.549020,...,0.000000,0.45098,0.009804,0.000000,0.0,0.0,0.0,0.539216,0.0,0.000000
4,12351.0,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,1.0,0.000000


#3. Feature Set 3: Product Category Preferences

In [21]:
def calculate_category_preferences(df):
    """
    Analyzes product category preferences for each customer.
    """
    # Extract product categories (first word of Description as a simple approach)
    df['Category'] = df['Description'].str.split().str[0]

    # Get top 20 categories by frequency
    top_categories = df['Category'].value_counts().nlargest(20).index

    # Filter only rows with top categories
    category_data = df[df['Category'].isin(top_categories)]

    # One-hot encode categories per customer
    category_pivot = pd.crosstab(category_data['Customer ID'], category_data['Category'])

    # Rename columns to have a consistent prefix
    category_pivot.columns = [f'Cat_{col}' for col in category_pivot.columns]

    # Calculate category diversity (number of unique categories purchased)
    cat_diversity = df.groupby('Customer ID')['Category'].nunique().reset_index()
    cat_diversity.columns = ['Customer ID', 'CategoryDiversity']

    # Combine category preferences
    category_prefs = category_pivot.reset_index().merge(cat_diversity, on='Customer ID')

    return category_prefs

# Calculate category preferences
category_preference_features = calculate_category_preferences(df_clean)
# Display the category preferences for a few customers
print(f"Category preferences extracted with {category_preference_features.shape[1] - 1} features")
category_preference_features.head()


Category preferences extracted with 21 features


,Customer ID,Cat_ASSORTED,Cat_BLUE,Cat_CHRISTMAS,Cat_DOOR,Cat_FELTCRAFT,Cat_HEART,Cat_JUMBO,Cat_LUNCH,Cat_PACK,...,Cat_RED,Cat_RETRO,Cat_SET,Cat_SMALL,Cat_STRAWBERRY,Cat_TEA,Cat_VINTAGE,Cat_WHITE,Cat_WOODEN,CategoryDiversity
0,12346.0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,4
1,12347.0,0,3,0,0,0,1,1,0,1,...,3,0,2,1,0,1,0,1,0,47
2,12348.0,0,0,0,0,0,0,0,0,8,...,0,0,3,0,0,0,0,0,0,9
3,12349.0,1,0,0,5,0,1,0,1,1,...,11,2,3,0,2,0,1,1,0,53
4,12351.0,0,1,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,16


#4. Feature Set 4: Purchase Variability and Basket Analysis

    Calculates metrics related to purchase patterns and basket characteristics.
    
    Purpose:
    - Measures consistency in shopping behavior
    - Analyzes shopping basket composition
    - Identifies price sensitivity
    

In [22]:
import numpy as np


In [23]:
# Calculate metrics related to purchase variability and basket characteristics
# This reveals shopping consistency and basket behaviors

def calculate_purchase_variability(df):
    """
    Calculates metrics related to purchase patterns and basket characteristics.

    Purpose:
    - Measures consistency in shopping behavior
    - Analyzes shopping basket composition
    - Identifies price sensitivity
    """
    # Calculate average basket size (items per transaction)
    basket_items = df.groupby(['Customer ID', 'Invoice'])['Quantity'].sum().reset_index()
    avg_basket = basket_items.groupby('Customer ID')['Quantity'].mean().reset_index()
    avg_basket.columns = ['Customer ID', 'AvgBasketSize']

    # Calculate basket size variability
    basket_var = basket_items.groupby('Customer ID')['Quantity'].std().reset_index()
    basket_var.columns = ['Customer ID', 'BasketSizeVariability']
    basket_var = basket_var.fillna(0)  # Fill NaN for customers with only one purchase

    # Calculate price sensitivity (average unit price paid)
    price_sensitivity = df.groupby('Customer ID')['Price'].mean().reset_index()
    price_sensitivity.columns = ['Customer ID', 'AvgUnitPrice']

    # Calculate purchase intervals
    # First get the first purchase date per invoice for each customer
    invoice_dates = df.groupby(['Customer ID', 'Invoice'])['InvoiceDate'].min().reset_index()
    invoice_dates = invoice_dates.sort_values(['Customer ID', 'InvoiceDate'])

    # Function to calculate intervals between purchases
    def get_purchase_intervals(group):
        if len(group) <= 1:
            return 0, 0  # No interval for single purchase

        # Calculate days between consecutive purchases
        group = group.sort_values('InvoiceDate')
        intervals = group['InvoiceDate'].diff().dt.days.dropna().tolist()

        if not intervals:
            return 0, 0

        return np.mean(intervals), np.std(intervals) if len(intervals) > 1 else 0

    # Apply the function to each customer group
    purchase_intervals = []
    for customer_id, group in invoice_dates.groupby('Customer ID'):
        mean_interval, std_interval = get_purchase_intervals(group)
        purchase_intervals.append({
            'Customer ID': customer_id,
            'PurchaseInterval': mean_interval,
            'PurchaseIntervalStd': std_interval
        })

    purchase_intervals_df = pd.DataFrame(purchase_intervals)

    # Combine all variability metrics
    purchase_variability = avg_basket.merge(basket_var, on='Customer ID', how='left')\
                                    .merge(price_sensitivity, on='Customer ID', how='left')\
                                    .merge(purchase_intervals_df, on='Customer ID', how='left')

    # Fill NaN values
    purchase_variability = purchase_variability.fillna(0)

    return purchase_variability

# Calculate purchase variability metrics
purchase_variability_features = calculate_purchase_variability(df_clean)

# Display the purchase variability for a few customers
print(f"Purchase variability features calculated")
purchase_variability_features.head()

Purchase variability features calculated


,Customer ID,AvgBasketSize,BasketSizeVariability,AvgUnitPrice,PurchaseInterval,PurchaseIntervalStd
0,12346.0,6.363636,4.653444,6.253333,19.2,34.778729
1,12347.0,414.000000,134.350288,2.295070,37.0,0.000000
2,12348.0,373.000000,0.000000,0.719500,0.0,0.000000
3,12349.0,331.000000,286.712399,8.581765,90.0,72.000000
4,12351.0,261.000000,0.000000,2.355238,0.0,0.000000


#5. Feature Set 5: Customer Longevity

    Calculates customer longevity-related metrics.
    
    Purpose:
    - Identifies customer lifecycle stage (new vs. established)
    - Measures customer retention timespan


In [28]:
def calculate_customer_longevity(df):
    """
    Calculates the number of days between a customer's first and last purchase.
    """
    # Find first and last purchase dates based on InvoiceDate
    first_purchase = df.groupby('Customer ID')['InvoiceDate'].min().reset_index()
    first_purchase.columns = ['Customer ID', 'FirstPurchase']

    last_purchase = df.groupby('Customer ID')['InvoiceDate'].max().reset_index()
    last_purchase.columns = ['Customer ID', 'LastPurchase']

    # Merge and calculate lifespan
    dates = first_purchase.merge(last_purchase, on='Customer ID')
    dates['CustomerLifespan'] = (dates['LastPurchase'] - dates['FirstPurchase']).dt.days

    return dates[['Customer ID', 'CustomerLifespan']]
# Calculate longevity metrics
longevity_features = calculate_customer_longevity(df_clean)

# Display result
print(f"Customer longevity calculated for {len(longevity_features)} customers")
longevity_features.head()


Customer longevity calculated for 4314 customers


,Customer ID,CustomerLifespan
0,12346.0,196
1,12347.0,37
2,12348.0,0
3,12349.0,181
4,12351.0,0


# Combine All Feature Sets and Normalize

In [25]:
'rfm_features' in locals()


True

In [29]:
# Combine all the feature sets into customer vectors
# This creates the final representation for similarity search

# Merge all feature sets
customer_features = rfm_features.merge(time_pattern_features, on='Customer ID', how='left')\
                      .merge(category_preference_features, on='Customer ID', how='left')\
                      .merge(purchase_variability_features, on='Customer ID', how='left')\
                      .merge(longevity_features, on='Customer ID', how='left')

# Fill missing values with appropriate defaults
customer_features = customer_features.fillna(0)

# Display the final feature set dimensions
print(f"Created feature vectors for {len(customer_features)} customers with {customer_features.shape[1] - 1} features")
print(f"Feature columns: {customer_features.columns.tolist()}")
customer_features.head()

Created feature vectors for 4314 customers with 54 features
Feature columns: ['Customer ID', 'Recency', 'Frequency', 'Monetary', 'AvgOrderValue', 'DayOfWeek_0', 'DayOfWeek_1', 'DayOfWeek_2', 'DayOfWeek_3', 'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6', 'TimeOfDay_Night', 'TimeOfDay_Morning', 'TimeOfDay_Afternoon', 'TimeOfDay_Evening', 'Month_1', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12', 'Cat_ASSORTED', 'Cat_BLUE', 'Cat_CHRISTMAS', 'Cat_DOOR', 'Cat_FELTCRAFT', 'Cat_HEART', 'Cat_JUMBO', 'Cat_LUNCH', 'Cat_PACK', 'Cat_PAPER', 'Cat_PINK', 'Cat_RED', 'Cat_RETRO', 'Cat_SET', 'Cat_SMALL', 'Cat_STRAWBERRY', 'Cat_TEA', 'Cat_VINTAGE', 'Cat_WHITE', 'Cat_WOODEN', 'CategoryDiversity', 'AvgBasketSize', 'BasketSizeVariability', 'AvgUnitPrice', 'PurchaseInterval', 'PurchaseIntervalStd', 'CustomerLifespan']


,Customer ID,Recency,Frequency,Monetary,AvgOrderValue,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,...,Cat_VINTAGE,Cat_WHITE,Cat_WOODEN,CategoryDiversity,AvgBasketSize,BasketSizeVariability,AvgUnitPrice,PurchaseInterval,PurchaseIntervalStd,CustomerLifespan
0,12346.0,164,11,372.86,11.298788,0.727273,0.151515,0.0,0.030303,0.090909,...,0.0,0.0,0.0,4.0,6.363636,4.653444,6.253333,19.2,34.778729,196
1,12347.0,2,2,1323.32,18.638310,0.000000,0.436620,0.0,0.000000,0.000000,...,0.0,1.0,0.0,47.0,414.000000,134.350288,2.295070,37.0,0.000000,37
2,12348.0,73,1,222.16,11.108000,1.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,9.0,373.000000,0.000000,0.719500,0.0,0.000000,0
3,12349.0,42,3,2671.14,26.187647,0.000000,0.009804,0.0,0.990196,0.000000,...,1.0,1.0,0.0,53.0,331.000000,286.712399,8.581765,90.0,72.000000,181
4,12351.0,10,1,300.93,14.330000,1.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,16.0,261.000000,0.000000,2.355238,0.0,0.000000,0


## Milestone 4 (Aya)
Large-Scale Hypothesis Testing: Conduct hypothesis testing across patient
subgroups to uncover statistically significant health factors. Apply corrections for multiple
testing (e.g., Benjamini-Hochberg).

##Milestone 5 (Louzali)
Recommendation Systems: Develop a recommendation engine to propose
lifestyle changes and follow-up actions based on predicted risks and matched patient
cohorts.